In [8]:
import itertools

#classe que representa um grafo valorado não orientado
class WeightedGraph:
    def __init__(self):
        self.graph = {} #listas de adjacencia
        self.N_vertices = 0
        self.N_edges = 0

    #adiciona um vértice ao grafo, caso ele ainda não tenha sido adicionado
    def add_vertex(self, vertex):
        if (vertex not in self.graph):
            self.graph[vertex] = {} #lista de adjacencia do vétice
            self.N_vertices = self.N_vertices+1

    #adiciona uma aresta entre dois vértices do grafo
    def add_edge(self, vertex1, vertex2, weight):
        if (vertex1 in self.graph and vertex2 in self.graph):
            if(weight >= 0):
                self.graph[vertex1][vertex2] = weight
                self.graph[vertex2][vertex1] = weight  # para um grafo não direcionado
            
            #distâncias negativas significam que não há caminho
            else:
                self.graph[vertex1][vertex2] = float('inf')
                self.graph[vertex2][vertex1] = float('inf')
            self.N_edges = self.N_edges+1
    
    #remove uma aresta entre dois vértices do grafo
    def remove_edge(self, vertex1, vertex2):
        if (vertex1 in self.graph and vertex2 in self.graph):
            del self.graph[vertex1][vertex2]
            del self.graph[vertex2][vertex1]# para um grafo não direcionado
            self.N_edges = self.N_edges-1 

    #printa o grafo como uma lista de adjacencias
    def print(self):
        for vertex, neighbors in self.graph.items():
            neighbor_str = ", ".join([f"{neighbor} ({weight})" for neighbor, weight in neighbors.items()])
            print(f"{vertex}: {neighbor_str}")

    #lê um grafo a partir de um arquivo
    def read_from_file(self, file_path):
            with open(file_path, 'r') as file:
                num_vertices = int(file.readline().strip())
                num_edges = int(file.readline().strip())

                for _ in range(num_vertices):
                    vertex = file.readline().strip()
                    graph.add_vertex(vertex)

                for _ in range(num_edges):
                    line = file.readline().strip()
                    vertex1, vertex2, weight = line.split()
                    vertex1 = str(vertex1)
                    vertex2 = str(vertex2)
                    weight = float(weight)
                    graph.add_edge(vertex1, vertex2, weight)
                #linhas após o termino das arestas são ignoradas

    #resolve o problema do caixeiro viajante por força bruta
    #retorna o menor caminho e o comprimento desse caminho
    def tsp_bruteforce(self):
        vertices = list(self.graph.keys())

        shortest_path = []
        shortest_distance = float('inf')

        if self.N_vertices < 2:
            return shortest_path, 0  #um único vértice não configura caminho 

        #lista cada permutação possível dos vértices
        for permuted_vertices in itertools.permutations(vertices):
            total_distance = 0
            for i in range(self.N_vertices - 1):
                vertex1 = permuted_vertices[i]
                vertex2 = permuted_vertices[i + 1]
                total_distance += self.graph[vertex1][vertex2]

            # soma as distancias do primeiro ao ultimo vertice
            total_distance += self.graph[permuted_vertices[-1]][permuted_vertices[0]]

            if total_distance < shortest_distance:#escolhe a menor
                shortest_distance = total_distance
                shortest_path = list(permuted_vertices)

        return shortest_path, shortest_distance

if __name__ == "__main__":

    graph = WeightedGraph()

    PATH = "data/euclidian.txt"

    graph.read_from_file(PATH)

    graph.print()

    shortest_path, shortest_distance = graph.tsp_bruteforce()

    print("Caminho mais curto:", shortest_path)
    print("Comprimento total", shortest_distance)

M: V (inf), T (inf), J (inf), S (inf), U (inf), N (inf)
V: M (inf), T (inf), J (inf), S (inf), U (inf), N (inf)
T: M (inf), V (inf), J (inf), S (inf), U (inf), N (inf)
J: M (inf), V (inf), T (inf), S (inf), U (inf), N (inf)
S: M (inf), V (inf), T (inf), J (inf), U (inf), N (inf)
U: M (inf), V (inf), T (inf), J (inf), S (inf), N (inf)
N: M (inf), V (inf), T (inf), J (inf), S (inf), U (inf)
Caminho mais curto: []
Comprimento total inf
